# Lab 3: Three NLP tasks

The assignment covers three NLP tasks related to the language structure and meaning.  
Part 1 is about probabilistic context-free grammar parsing, a task of recovering the underlying syntactic structure of a sentence using a probabilistic grammar.  
Part 2 deals with word meaning as it tackles the task of identifying the correct sense of a word in the context.  
Part 3 concerns reasoning with math question-answering problems and examines it from the generalization and explainability perspectives.

Shall we begin?

# Rules

You are greatly encouraged to add comments to your code describing what particular lines of code do (in general, a great habit to have in your coding life).
Additionally, please follow these rules when submitting the notebook:

* Put all code in the cell with the `# YOUR CODE HERE` comment.
* For theoretical questions, put your solution in the `YOUR ANSWER HERE` or `ANSWER UNDER THIS LINE` cells (and keep the header if any).
* Don't change or delete any initially provided cells, either text or code, unless explicitly instructed to do so.
* Don't delete the comment lines `# TEST...` or edit their code cells. The test cells are for sanity checking. Passing them doesn't necessarily mean that your code is fine.
* Don't change the names of provided functions and variables or arguments of the functions.
* Don't clear the output of your code cells.
* Don't output unnecessary info (e.g., printing variables for debugging purposes). This clutters the notebook and slows down the grading. You can have print() in the code, but comment them out before submitting the notebook.
* Delete those cells that you inserted for your own debugging/testing purposes.
* Don't forget to fill in the **contribution information**.
* Don't forget to fill in the **work description section** per exercise.
* Test your code and **make sure we can run your notebook** in the colab environment.
* A single notebook file (without archiving) per group should be submitted via BB.

<font color="red">Following these rules helps us to grade the submissions relatively efficiently. If these rules are violated, a submission will be subject to penalty points.</font>  

# <font color="red">Contributions</font>

YOUR ANSWER HERE [30-50 words]

**Names:**

- Alexia Ntantouri
- Juan David Liut Aymar

**Contributions:**

**Part 1:** Code by David, review by Alexia<br>

**Part 2:** Code and review by Alexia and David<br>

**Part 3:** Code by Alexia, review by David<br>

# General instructions

Before diving into the exercises, keep in mind that the variables defined previously can be reused in the subsequent cells. So there is no need to redefine the same variable in multiple sections, e.g., it is sufficient to read the file in a variable once and later reuse the value of the variable, instead of re-reading the file.   

If your code is too long and uses several code cells instead of a single code cell, rethink how to organize data in variables so that you can easily access the required info. Reading about [list comprehension](https://realpython.com/list-comprehension-python/#leverage-list-comprehensions) can be useful.

Your code will often be evaluated based on its behaviour. So, during the grading some code cells are executed. If code runtime is too long than expected, this will hinder grading.

<font color="red">**The cases similar to the above-mentioned ones will be subject to penalty points.**</font>

<font color="red">**Pay attention to test units**</font> that are either provided as assert cases or as comments. Test units help you by giving you a hint about the correct answer. Note that **passing test units doesn't guarantee the full points** for an exercise because test units are incomplete, and the code might fail on other test units.

# Part 1: Constituency parsing with CKY

The grammatical structure of a sentence can be represented with a Context Free Grammar (CFG). When we additionally assign probabilities to the rules of the CFG we get a PCFG: a _Probabilistic_ CFG.

Given a sufficiently expressive PCFG (one that holds enough rules) we can parse new sentences using the Cocke–Kasami–Younger (CKY) algorithm. You can use this algorithm in three ways: to find the set of all the possible parses $p$ of a sentence $s$ under a PCFG $G$; to find the probability of the sentence by summing up the probabilities of these parses; or to find the parse $p^{*}$ of the highest probability.


### Tasks
1. In this notebook you will learn how to represent a PCFG in an object-oriented manner as a collection of python classes. These classes are already defined for you. Read them through thoroughly and make sure that you understand them well. You have to use them when implementing the CKY algorithm.

2. Implement the CKY algorithm to find the most probable parse $p^{*}$ for a sentence. Your implementation will follow the psuedo-code that is given in both the lecture slides, and Jurafsky and Martin.

## Set up

In [ ]:
import numpy as np
from collections import Counter, defaultdict
import math
# nltk will be used to draw constituency parses
import nltk
from nltk.tree import Tree

In [ ]:
# downloading grammar files
! wget -nv https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-1.txt
! wget -nv https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-2.txt
! wget -nv https://naturallogic.pro/_files_/download/mNLP/telescope-grammar.txt

2025-06-08 09:09:07 URL:https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-1.txt [337/337] -> "groucho-grammar-1.txt" [1]
2025-06-08 09:09:07 URL:https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-2.txt [337/337] -> "groucho-grammar-2.txt" [1]
2025-06-08 09:09:08 URL:https://naturallogic.pro/_files_/download/mNLP/telescope-grammar.txt [381/381] -> "telescope-grammar.txt" [1]


## PCFG

In this lab we will show you a way to represent a **PCFG** using python objects. We will introduce the following classes:

* Symbol
    * Terminal
    * Nonterminal
* Rule

At first glance, this might seem like a lot of work. But, hopefully, by the time you get to implementing CKY you will be convinced in the benefits of these constructions.

### Symbol

Recall that:
* **Terminal** symbols are the words of the sentence: _I, ate, salad, the_ etc.
* **Nonterminal** symbols are the syntactic categories of the various constituents: _S, NP, VP, Det_ etc.

In our representation, `Symbol` is going to be a container class. The classes `Terminal` and `Nonterminal` will *inherit* from the `Symbol` class and will hence both become a type of symbol. The classes themselves are effectively a container for the underlying python strings.

In [ ]:
class Symbol:
    """
    A symbol in a grammar.
    This class will be used as parent class for Terminal, Nonterminal.
    This way both will be a type of Symbol.
    """
    def __init__(self):
        pass


class Terminal(Symbol):
    """
    Terminal symbols are words in a vocabulary

    E.g. 'I', 'ate', 'salad', 'the'
    """

    def __init__(self, symbol: str):
        assert type(symbol) is str, f"A Terminal takes a python string, got {type(symbol)}"
        self._symbol = symbol

    def is_terminal(self):
        return True

    def is_nonterminal(self):
        return False

    def __str__(self):
        return f"'{self._symbol}'"

    def __repr__(self):
        return f"Terminal({repr(self._symbol)})"

    def __hash__(self):
        return hash(self._symbol)

    def __len__(self):
        """The length of the underlying python string"""
        return len(self._symbol)

    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self._symbol < other._symbol

    @property
    def obj(self):
        """Returns the underlying python string"""
        return self._symbol


class Nonterminal(Symbol):
    """
    Nonterminal symbols are the grammatical classes in a grammar.

    E.g. S, NP, VP, N, Det, etc.
    """

    def __init__(self, symbol: str):
        assert type(symbol) is str, f"A Nonterminal takes a python string, got {type(symbol)}"
        self._symbol = symbol

    def is_terminal(self):
        return False

    def is_nonterminal(self):
        return True

    def __str__(self):
        return f"[{self._symbol}]"

    def __repr__(self):
        return f"Nonterminal({repr(self._symbol)})"

    def __hash__(self):
        return hash(self._symbol)

    def __len__(self):
        """The length of the underlying python string"""
        return len(self._symbol)

    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self._symbol < other._symbol

    @property
    def obj(self):
        """Returns the underlying python string"""
        return self._symbol

Let's try out the classes by initializing some terminal and nonterminal symbols:

In [ ]:
dog = Terminal('dog')
the = Terminal('the')
walks = Terminal('walks')

S = Nonterminal('S')
NP = Nonterminal('NP')
NP_prime = Nonterminal('NP')
VP = Nonterminal('VP')
V = Nonterminal('V')
N = Nonterminal('N')
Det = Nonterminal('Det')

The methods `__eq__` and `__ne__` make it possible to compare our objects using standard Python syntax. But more importantly: compare in the way that we are interested in, namely whether the underlying representation is the same.

To see the difference, try commenting out the method `__eq__` in the class above, and notice the different result of the equality test `NP==NP_prime`.

In [ ]:
print(dog)
print(NP)
print(NP==Det)
print(NP!=Det)
print(NP==NP)
print(NP==NP_prime)

'dog'
[NP]
False
True
True
True


Note the difference between calling `print(NP)` and simply calling `NP`. The first is taken care of by the method `__str__` and the second by the method `__repr__`.

In [ ]:
dog

Terminal('dog')

We can also easily check if our symbol is a terminal or not:

In [ ]:
dog.is_terminal()

True

In [ ]:
NP.is_terminal()

False

Finally the method `__hash__` makes our object *hashable*, and hence usable in a datastructure like a dictionary.

Try commenting out this method above in the class and then retry constructing the dictionary: notice the error.

In [ ]:
d = {NP: 1, S: 2}
d

{Nonterminal('NP'): 1, Nonterminal('S'): 2}

### Rules

In a PCFG a **rule** looks something like this

$$NP \to Det\;N$$

with a corresponding probability, for example $1.0$ if we lived in a world where all noun phrases had this grammatical structure.

In our representation, `Rule` will be an object made of a left-hand side (`lhs`) symbol, a sequence of right-hand side symbols (`rhs`) and a probability `prob`.

If we use the above defined symbols, we can call

    rule = Rule(NP, [Det, N], 1.0)
   
This will construct an instance called `rule` which represent the rule above

    [NP] -> [Det] [N] (1.0)
    

In [ ]:
class Rule:

    def __init__(self, lhs, rhs, prob):
        """
        Constructs a Rule.
        A Rule takes a LHS symbol and a list/tuple of RHS symbols.

        :param lhs: the LHS nonterminal
        :param rhs: a sequence of RHS symbols (terminal or nonterminal)
        :param prob: probability of the rule
        """

        assert isinstance(lhs, Symbol), 'LHS must be an instance of Symbol (actually even a non-terminal but later we will expan LHS)'
        assert len(rhs) > 0, 'If you want an empty RHS, use an epsilon Terminal EPS'
        assert all(isinstance(s, Symbol) for s in rhs), 'RHS must be a sequence of Symbol objects'
        if prob is not None:
            assert 0 <= prob <= 1, 'The probability must be between 0 and 1'
        self._lhs = lhs
        self._rhs = tuple(rhs)
        self._prob = prob


    def __eq__(self, other):
        return self._lhs == other._lhs and self._rhs == other._rhs and self._prob == other._prob

    def __ne__(self, other):
        return not (self == other)

    def __hash__(self):
        return hash((self._lhs, self._rhs, self._prob))

    def __repr__(self):
        rhs = ' '.join(str(sym) for sym in self._rhs)
        return f"{self._lhs} -> {rhs} ({self.prob})"

    def is_binary(self):
        """True if Rule is binary: A -> B C"""
        return len(self._rhs) == 2

    def is_unary(self):
        """True if Rule is unary: A -> w"""
        return len(self._rhs) == 1

    @property
    def lhs(self):
        """Returns the lhs of the rule"""
        return self._lhs

    @property
    def rhs(self):
        """Returns the rhs of the rule"""
        return self._rhs

    @property
    def prob(self):
        """Returns the probability of the rule"""
        return self._prob


Just as with `Terminal` and `Nonterminal` you can print an instance of `Rule`, you can access its attributes, and you can hash rules with containers such as dict and set.

In [ ]:
r1 = Rule(S, [NP, VP], 1.0)
r2 = Rule(NP, [Det, N], 1.0)
r3 = Rule(N, [dog], 1.0)
r4 = Rule(Det, [the], 1.0)
r5 = Rule(VP, [walks], 1.0)

print(r1)
print(r2)
print(r3)
print(r4)

[S] -> [NP] [VP] (1.0)
[NP] -> [Det] [N] (1.0)
[N] -> 'dog' (1.0)
[Det] -> 'the' (1.0)


In [ ]:
print(r1.prob)

1.0


In [ ]:
r1 in set([r1])

True

In [ ]:
d = {r1: 1, r2: 2}
d

{[S] -> [NP] [VP] (1.0): 1, [NP] -> [Det] [N] (1.0): 2}

### Grammar

A `PCFG` class is a container for `Rules`. The `Rules` are stored in the `PCFG` in such a way that they can be accesed easily in different ways.

In [ ]:
class PCFG(object):
    """
    Constructs a PCFG.
    A PCFG stores a list of rules that can be accessed in various ways.

    :param rules: an optional list of rules to initialize the grammar with
    """
    def __init__(self, rules=[]):
        self._rules = []
        self._rules_by_lhs = defaultdict(list)
        self._terminals = set()
        self._nonterminals = set()
        for rule in rules:
            self.add(rule)

    def add(self, rule):
        """Adds a rule to the grammar"""
        if not rule in self._rules:
            self._rules.append(rule)
            self._rules_by_lhs[rule.lhs].append(rule)
            self._nonterminals.add(rule.lhs)
            for s in rule.rhs:
                if s.is_terminal():
                    self._terminals.add(s)
                else:
                    self._nonterminals.add(s)

    def update(self, rules):
        """Add a list of rules to the grammar"""
        for rule in rules:
            self.add(rule)

    @property
    def nonterminals(self):
        """The list of nonterminal symbols in the grammar"""
        return self._nonterminals

    @property
    def terminals(self):
        """The list of terminal symbols in the grammar"""
        return self._terminals

    @property
    def rules(self):
        """The list of rules in the grammar"""
        return self._rules

    @property
    def binary_rules(self):
        """The list of binary rules in the grammar"""
        return [rule for rule in self._rules if rule.is_binary()]

    @property
    def unary_rules(self):
        """The list of unary rules in the grammar"""
        return [rule for rule in self._rules if rule.is_unary()]

    def __len__(self):
        return len(self._rules)

    def get(self, lhs):
        """The list of rules whose LHS is the given symbol lhs"""
        return self._rules_by_lhs.get(lhs, [])

    def __iter__(self):
        """Iterator over rules (in arbitrary order)"""
        return iter(self._rules)

    def iteritems(self):
        """Iterator over pairs of the kind (LHS, rules rewriting LHS)"""
        return self._rules_by_lhs.items()

    def __str__(self):
        """Prints the grammar line by line"""
        lines = []
        for lhs, rules in self.iteritems():
            for rule in rules:
                lines.append(str(rule))
        return '\n'.join(lines)

Initialize a grammar

In [ ]:
G = PCFG()

We can add rules individually with `add`, or as a list with `update`:

In [ ]:
G.add(r1)
G.update([r2,r3,r4,r5])

We can print the grammar

In [ ]:
print(G)

[S] -> [NP] [VP] (1.0)
[NP] -> [Det] [N] (1.0)
[N] -> 'dog' (1.0)
[Det] -> 'the' (1.0)
[VP] -> 'walks' (1.0)


We can get the set of rewrite rules for a certain LHS symbol.

In [ ]:
G.get(S)

[[S] -> [NP] [VP] (1.0)]

In [ ]:
G.get(NP)

[[NP] -> [Det] [N] (1.0)]

We can also iterate through rules in the grammar.

Note that the following is basically counting how many rules we have in the grammar.

In [ ]:
sum(1 for r in G)

5

which can also be done in a more concise way

In [ ]:
len(G)

5

We can access the set of terminals and nonterminals of the grammar:

In [ ]:
print(G.nonterminals)

{Nonterminal('NP'), Nonterminal('VP'), Nonterminal('Det'), Nonterminal('N'), Nonterminal('S')}


In [ ]:
print(G.terminals)

{Terminal('the'), Terminal('walks'), Terminal('dog')}


In [ ]:
S in G.nonterminals

True

In [ ]:
dog in G.terminals

True

Finally we can easily access all the binary rules and all the unary rules in the grammar:

In [ ]:
G.unary_rules

[[N] -> 'dog' (1.0), [Det] -> 'the' (1.0), [VP] -> 'walks' (1.0)]

In [ ]:
G.binary_rules

[[S] -> [NP] [VP] (1.0), [NP] -> [Det] [N] (1.0)]

## Visualizing a tree

For the sake of legacy let's reiterate an age-old NLP schtick, the well-known example of structural ambiguity from the Groucho Marx movie, [Animal Crackers](https://youtu.be/FZUfhfHbjE4?t=1m33s) (1930):

> One morning I shot an elephant in my pajamas. How he got into my pajamas, I don't know.

Let's take a closer look at the ambiguity in the phrase: _I shot an elephant in my pajamas_. The ambiguity is caused by the fact that the sentence has two competing parses represented in:

    (S (NP I) (VP (VP (V shot) (NP (Det an) (N elephant))) (PP (P in) (NP (Det my) (N pajamas)))))

and

    (S (NP I) (VP (V shot) (NP (Det an) (NP (N elephant) (PP (P in) (NP (Det my) (N pajamas)))))))


We can write these parses down as strings and then let NLTK turn them into trees using the NLTK `Tree` class. (See http://www.nltk.org/api/nltk.html#nltk.tree.Tree as reference for this class, if you want to know more.)

In [ ]:
parse1 = "(S (NP I) (VP (VP (V shot) (NP (Det an) (N elephant))) (PP (P in) (NP (Det my) (N pajamas)))))"
parse2 = "(S (NP I) (VP (V shot) (NP (Det an) (NP (N elephant) (PP (P in) (NP (Det my) (N pajamas)))))))"

pajamas1 = Tree.fromstring(parse1)
pajamas2 = Tree.fromstring(parse2)

We can then *pretty-print* these trees:

In [ ]:
pajamas1.pretty_print()
pajamas2.pretty_print()

     S                                       
  ___|______________                          
 |                  VP                       
 |         _________|__________               
 |        VP                   PP            
 |    ____|___              ___|___           
 |   |        NP           |       NP        
 |   |     ___|_____       |    ___|_____     
 NP  V   Det        N      P  Det        N   
 |   |    |         |      |   |         |    
 I  shot  an     elephant  in  my     pajamas

     S                                       
  ___|__________                              
 |              VP                           
 |    __________|______                       
 |   |                 NP                    
 |   |     ____________|___                   
 |   |    |                NP                
 |   |    |      __________|___               
 |   |    |     |              PP            
 |   |    |     |       _______|___           
 |   |    |     |      

## Parsing with CKY

Let's stick with this sentence for the rest of this lab. We will use CKY to find the 'best' parse for this sentence.

In [ ]:
# Turn the sentence into a list
sentence = "I shot an elephant in my pajamas".split()
# The length of the sentence
num_words = len(sentence)

A PCFG for this sentence can be found in the file `groucho-grammar-1.txt`. We read this in with the function `read_grammar_rules`.

In [ ]:
def read_grammar_rules(istream):
    """Reads grammar rules formatted as 'LHS ||| RHS ||| PROB'."""
    for line in istream:
        line = line.strip()
        if not line: continue
        fields = line.split('|||')
        if len(fields) != 3:
            raise ValueError(f"Three fields were expected: {fields}")
        lhs = fields[0].strip()

        if lhs.startswith('[') and lhs.endswith(']'):
            lhs = Nonterminal(lhs[1:-1])
        else:
            raise ValueError(f"LHS must be a non-terminal: {fields}")
        rhs = fields[1].strip().split()
        new_rhs = []
        for r in rhs:
            if r.startswith('[') and r.endswith(']'):
                r = Nonterminal(r[1:-1])
            else:
                r = Terminal(r)
            new_rhs.append(r)

        prob = float(fields[2].strip())
        yield Rule(lhs, new_rhs, prob)

In [ ]:
# Read in the grammar
with open('groucho-grammar-1.txt') as F:
    grammar = PCFG(read_grammar_rules(F))
print("The grammar:\n", grammar)

The grammar:
 [S] -> [NP] [VP] (1.0)
[PP] -> [P] [NP] (1.0)
[NP] -> [Det] [N] (0.2)
[NP] -> [Det] [NP] (0.3)
[NP] -> [N] [PP] (0.3)
[NP] -> 'I' (0.2)
[VP] -> [V] [NP] (0.4)
[VP] -> [VP] [PP] (0.6)
[Det] -> 'an' (0.6)
[Det] -> 'my' (0.4)
[N] -> 'elephant' (0.5)
[N] -> 'pajamas' (0.5)
[V] -> 'shot' (1.0)
[P] -> 'in' (1.0)


We will also need the following two dictionaries: `n2i` mapping from nonterminals to integers (indices); and its inverse, an `i2n` dictionary.

In [ ]:
num_nonterminals = len(grammar.nonterminals)

n2i = defaultdict(lambda: len(n2i))
i2n = dict()

# sort nonterminals to make the mapping deterministic
for nt in sorted(grammar.nonterminals):
    i2n[n2i[nt]] = nt

# Stop defaultdict behavior of n2i
n2i = dict(n2i)

n2i

{Nonterminal('Det'): 0,
 Nonterminal('N'): 1,
 Nonterminal('NP'): 2,
 Nonterminal('P'): 3,
 Nonterminal('PP'): 4,
 Nonterminal('S'): 5,
 Nonterminal('V'): 6,
 Nonterminal('VP'): 7}

### The charts

Now we are ready to introduce the chart datastructures. We need a chart to store the **scores** and a chart to store the **backpointers**.

Both of these will be 3-dimensional numpy arrays: one named `score` holding the probabilities of intermediate results; one named `back` to store the backpointers in. We will use the following indexing convention for these charts:

* Format of the chart holding the **scores** `score[A][begin][end] = probability`.
This is interpreted as the probability of the constituent between `begin:end` being parsed with `A` as its root.      
         
* Format of the chart holding the **backpointers** `back[A][begin][end] = (split, B, C)`.
This is interpreted as the constituent `begin:end` can be combined with a rule `A -> B C` where `begin:split` is `B` and `split:end` is `C`.

This indexing convention is convenient for printing. See what happens when we print `back` below: we get `num_nonterminal` slices, each a numpy array of shape `[n_words+1, n_words+1]`. This is easier to read than the format `back[i][j][A]`.

**[Note]** Here we pretended `A` is both the nonterminal as well as the index. In our implementation `A` will be the nonterminal and the index for `A` will be `n2i[A]`.  

Let's show you what we mean:

In [ ]:
# A numpy array zeros
score = np.zeros((num_nonterminals,
                  num_words + 1,
                  num_words + 1))

# A numpy array that can store arbitrary data (we set dtype to object)
back = np.zeros((num_nonterminals,
                 num_words + 1,
                 num_words + 1), dtype=object)

The following illustrates the way you will use the `back` chart. In this example, your parser recognized that the entire sequence is S while the words between 0 and 2 form NP, and the words between 2 and the end of the sentence form VP (and nothing else yet):

In [ ]:
# Illustration of the backpointer array
back[n2i[S]][0][-1] = (2,NP,VP)
back

array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        

## Ex1.1 [10pt] CKY parsing

Implement the **CKY** algorithm. Follow the pseudo-code given in the lecture-slides (or alternatively in J&M). The code must comply to the following:

* The function `cky` takes a sentence (list of words), a grammar (an instance of PCFG), and a n2i non-terminals-to-index dictionary.
* The function `cky` returns the filled-in score-chart and backpointer-chart, following the format established above.
* No global variables should be accessed from the body of the function (except for the predefined classes).

**[Hint]** This is the moment to make good use of the methods of the classes `PCFG`, `Rule`, `Nonterminal`, and `Terminal`!

In [ ]:
# @title cky function
def cky(sentence, grammar, n2i):
    """
    The CKY algorithm.

    :param sentence: a list of words
    :param grammar: an instance of the class PCFG
    :param n2i: a dictionary mapping from Nonterminals to indices
    :return score: the filled in scores chart
    :return back: the filled in backpointers chart
    """
    num_words = len(sentence)
    num_nonterminals = len(grammar.nonterminals)

    # A numpy array to store the scores of intermediate parses
    score = np.zeros((num_nonterminals,
                  num_words + 1,
                  num_words + 1))

    # A numpy array to store the backpointers
    back = np.zeros((num_nonterminals,
                     num_words + 1,
                     num_words + 1), dtype=object)

    ## YOUR CODE HERE ##
    # 1. Initialize with lexical rules (A -> w)
    for i in range(num_words):
        word = sentence[i] # get the i-th word and treat it as a terminal
        terminal = Terminal(word)
        for rule in grammar.rules:
            if rule.is_unary() and rule.rhs[0] == terminal: # check for unary rules with terminal on the right
                A = rule.lhs
                A = rule.lhs
                A_idx = n2i[A]
                score[A_idx][i][i+1] = rule.prob # the substring from word i to i+1 can be generated using rule A
                back[A_idx][i][i+1] = None  # unary rules don't have backpointers

    # 2. Handle spans of increasing length
    for span in range(2, num_words + 1):  # length of the span
        for begin in range(num_words + 1 - span):
            end = begin + span
            for split in range(begin + 1, end):
                for rule in grammar.rules:
                    if rule.is_binary():
                        B, C = rule.rhs
                        A = rule.lhs
                        A_idx = n2i[A]
                        B_idx = n2i[B]
                        C_idx = n2i[C]
                        prob = rule.prob * score[B_idx][begin][split] * score[C_idx][split][end]
                        if prob > score[A_idx][begin][end]:
                            score[A_idx][begin][end] = prob
                            back[A_idx][begin][end] = (split, B, C)

    return score, back

In [ ]:
# Run CKY
score, back = cky(sentence, grammar, n2i)

### Check your CKY

Use the code in the following two cell to check your `cky` implementation.

Take the Nonterminal `S` to inspect your filled in score and backpointer charts. **Leave the code in this cell unchanged.** We will use this to evaluate the corectness of your cky function.

In [ ]:
# TEST EX1.1
### Don't change the code in this cell ###

S = Nonterminal('S')

print('The whole slice for nonterminal S:')
print(score[n2i[S]], "\n")

print('The score in cell (S, 0, num_words), which is the probability of the best parse:')
print(score[n2i[S]][0][num_words], "\n")

print('The backpointer in cell (S, 0, num_words):')
print(back[n2i[S]][0][num_words], "\n")

print(back[n2i[S]])

The whole slice for nonterminal S:
[[0.        0.        0.        0.        0.0048    0.        0.
  0.0001152]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.       ]] 

The score in cell (S, 0, num_words), which is the probability of the best parse:
0.00011520000000000004 

The backpointer in cell (S, 0, num_words):
(1, Nonterminal('NP'), Nonterminal('VP')) 

[[0 0 0 0 (1, Nonterminal('NP'), Nonterminal('VP')) 0 0
  (1, Nonterminal('NP'), Nonterminal('VP'))]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 

## Ex1.2 [5pt] Recovering a tree

Write the function `build_tree` that reconstructs the parse from the backpointer table. This is the function that is called in the return statement of the [pseudo-code](https://web.stanford.edu/~jurafsky/slp3/C.pdf#page=6) in Jurafsky and Martin.

**[Note]** We have no pseudocode for you here: you must come up with your own implementation. However we do provide you with the expected output so that you can at least partially test your code.

Here is some additional advice:

* Use recursion - that is write your function in a recursive way.  
What is the base case? Hint: $A \to w$.  
What is the recursive case? Hint: $A \to B\; C$.
    
    
* Use the additional class `Span` that we introduce below for the symbols in your recovered rules. Read the documentation in the `Span` class for its usage.
    
    
* In order to use the function `make_nltk_tree` (which we provide and that turns a `derivation` into an NLTK tree so that you can draw it), your function must return the <font color="red">**list of rules in derivation ordered [depth-first](https://en.wikipedia.org/wiki/Depth-first_search)**</font>. If you write your function recursively such order can be achieved easily.

The following class will be very useful in your solution for the function `build_tree`.    

In [ ]:
class Span(Symbol):
    """
    A Span indicates that symbol was recognized between begin and end.

    Example:
        Span(Terminal('the'), 0, 1)
            This means: we found 'the' in the sentence between 0 and 1
        Span(Nonterminal('NP'), 4, 8) represents NP:4-8
            This means: we found an NP that covers the part of the sentence between 4 and 8

    Thus, Span holds a Terminal or a Nonterminal and wraps it between two integers.
    This makes it possible to distinguish between two instances of the same rule in the derivation.
    Example:
        We can find that the rule NP -> Det N is used twice in the parse derivation. But that in the first
        case it spans "an elephant" and in the second case it spans "my pajamas". We want to distinguis these.
        So: "an elephant" is covered by [NP]:2-4 -> [Det]:2-3 [N]:3-4
            "my pajamas" is covered by [NP]:5-7 -> [Det]:5-6 [N]:6-7

    Internally, we represent spans with tuples of the kind (symbol, start, end).
    """

    def __init__(self, symbol, start, end):
        assert isinstance(symbol, Symbol), f"A span takes an instance of Symbol, got {type(symbol)}"
        self._symbol = symbol
        self._start = start
        self._end = end

    def is_terminal(self):
        # a span delegates this to an underlying symbol
        return self._symbol.is_terminal()

    def obj(self):
        """The underlying python tuple (Symbol, start, end)"""
        return (self._symbol, self._start, self._end)

    def __str__(self):
        """Prints Symbol surrounded with begin and end (purely aesthetics)"""
        return f"{self._start}:{self._symbol}:{self._end}"

    def __repr__(self):
        return f"Span({self._symbol!r}, {self._start!r}, {self._end!r})"

    def __hash__(self):
        return hash((self._symbol, self._start, self._end))

    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol and self._start == other._start and self._end == other._end

    def __ne__(self, other):
        return not (self == other)

Example usage of `Span`:

In [ ]:
span_S = Span(S, 0, 10)
print(span_S)
span_S = Span(dog, 4, 5)
print(span_S)

spanned_rule = Rule(Span(NP, 2, 4), [Span(Det, 2, 3), Span(NP, 3, 4)], prob=None)
print(spanned_rule)

0:[S]:10
4:'dog':5
2:[NP]:4 -> 2:[Det]:3 3:[NP]:4 (None)


Your final derivation should look like this:

```
[0:[S]:7 -> 0:[NP]:1 1:[VP]:7 (None),
 0:[NP]:1 -> 0:'I':1 (None),
 1:[VP]:7 -> 1:[VP]:4 4:[PP]:7 (None),
 1:[VP]:4 -> 1:[V]:2 2:[NP]:4 (None),
 1:[V]:2 -> 1:'shot':2 (None),
 2:[NP]:4 -> 2:[Det]:3 3:[N]:4 (None),
 2:[Det]:3 -> 2:'an':3 (None),
 3:[N]:4 -> 3:'elephant':4 (None),
 4:[PP]:7 -> 4:[P]:5 5:[NP]:7 (None),
 4:[P]:5 -> 4:'in':5 (None),
 5:[NP]:7 -> 5:[Det]:6 6:[N]:7 (None),
 5:[Det]:6 -> 5:'my':6 (None),
 6:[N]:7 -> 6:'pajamas':7 (None)]
 ```

(Note that the rule probabilities are set to `None`. These are not saved in the backpointer chart so cannot be retrieved at the recovering stage. They also don't matter at this point, so you can set them to `None`.)

In [ ]:
# @title build_tree function
def build_tree(back, sentence, root, n2i, begin=0, end=None):
    """
    Reconstruct the viterbi parse from a filled-in backpointer chart.
    The function takes detailed arguments to be easily used in a recursive way.

    It returns a list called derivation which holds the rules over Spans.
    In order to use the function make_nltk_tree for the output,
    you must make sure that the order in derivation follows the depth-first order (!!!).

    :param back: a backpointer chart of shape [num_nonterminals, num_words+1, num_words+1]
    :param sentence: a list of words
    :param root: the root symbol of the tree: usually Nonterminal('S')
    :param n2i: the dictionary mapping from Nonterminals to indices
    :param begin: index that marks the start of the target segment of the sentence
    :param end: index that marks the end of the target segment of the sentence
    :param n2i: the dictionary mapping from Nonterminals to indices
    :return derivation: a derivation: a list of Rules with Span symbols that generate the Viterbi tree.
                        The list should be ordered depth first!
    """
    if end is None: end = len(sentence)
    assert isinstance(end, int), "when end argument is specified, it needs to be an integer"
    derivation = []
    ### YOUR CODE HERE ###

    root_index = n2i[root]
    pointer = back[root_index][begin][end]

    lhs = Span(root, begin, end)

    if pointer == 0 or pointer is None:
        word = sentence[begin]
        rhs = [Span(Terminal(word), begin, end)]

        # Find the probability for the unary lexical rule
        prob = None
        for rule in grammar.rules:
            if rule.lhs == root and len(rule.rhs) == 1 and rule.rhs[0] == Terminal(word):
                prob = rule.prob
                break

        derivation.append(Rule(lhs, rhs, prob))
        return derivation

    split, B, C = pointer
    rhs = [Span(B, begin, split), Span(C, split, end)]

    # Add the parent rule first
    prob = None

    for rule in grammar.rules:
        if rule.lhs == root and list(rule.rhs) == [B, C]: # Convert rule.rhs into list before comparing because it was a tuple
            prob = rule.prob
            break
    derivation.append(Rule(lhs, rhs, prob))

    # Then recursively add the children
    derivation += build_tree(back, sentence, B, n2i, begin, split)
    derivation += build_tree(back, sentence, C, n2i, split, end)

    return derivation

Get your derivation:

In [ ]:
derivation = build_tree(back, sentence, S, n2i)
derivation

[0:[S]:7 -> 0:[NP]:1 1:[VP]:7 (1.0),
 0:[NP]:1 -> 0:'I':1 (0.2),
 1:[VP]:7 -> 1:[VP]:4 4:[PP]:7 (0.6),
 1:[VP]:4 -> 1:[V]:2 2:[NP]:4 (0.4),
 1:[V]:2 -> 1:'shot':2 (1.0),
 2:[NP]:4 -> 2:[Det]:3 3:[N]:4 (0.2),
 2:[Det]:3 -> 2:'an':3 (0.6),
 3:[N]:4 -> 3:'elephant':4 (0.5),
 4:[PP]:7 -> 4:[P]:5 5:[NP]:7 (1.0),
 4:[P]:5 -> 4:'in':5 (1.0),
 5:[NP]:7 -> 5:[Det]:6 6:[N]:7 (0.2),
 5:[Det]:6 -> 5:'my':6 (0.4),
 6:[N]:7 -> 6:'pajamas':7 (0.5)]

### Draw the tree

Turn the derivation into an NLTK tree:



In [ ]:
def make_nltk_tree(derivation):
    """
    Return a NLTK Tree object based on the derivation
    (list or tuple of Rules)
    """
    d = defaultdict(None, ((r.lhs, r.rhs) for r in derivation))

    def make_tree(lhs):
        return Tree(str(lhs), (str(child) if child not in d else make_tree(child) for child in d[lhs]))

    return make_tree(derivation[0].lhs)

If you give the derivation to the function `make_nltk_tree` and let NLTK draw it, then you get this tree:

```
          0:[S]:7                                                                              
    _________|_______________________________                                                   
   |                                      1:[VP]:7                                             
   |                     ____________________|_____________________                             
   |                 1:[VP]:4                                   4:[PP]:7                       
   |          __________|________                         _________|________                    
   |         |                2:[NP]:4                   |               5:[NP]:7              
   |         |           ________|___________            |          ________|___________        
0:[NP]:1  1:[V]:2   2:[Det]:3             3:[N]:4     4:[P]:5  5:[Det]:6             6:[N]:7   
   |         |          |                    |           |         |                    |       
0:'I':1  1:'shot':2  2:'an':3          3:'elephant':4 4:'in':5  5:'my':6          6:'pajamas':7
```

In [ ]:
# TEST EX1.2
tree = make_nltk_tree(derivation)
tree.pretty_print()

          0:[S]:7                                                                              
    _________|_______________________________                                                   
   |                                      1:[VP]:7                                             
   |                     ____________________|_____________________                             
   |                 1:[VP]:4                                   4:[PP]:7                       
   |          __________|________                         _________|________                    
   |         |                2:[NP]:4                   |               5:[NP]:7              
   |         |           ________|___________            |          ________|___________        
0:[NP]:1  1:[V]:2   2:[Det]:3             3:[N]:4     4:[P]:5  5:[Det]:6             6:[N]:7   
   |         |          |                    |           |         |                    |       
0:'I':1  1:'shot':2  2:'an':3      

## That's it!

Congratulations, you have made it to the end of the lab.

**Make sure all your cells are executed so that all your answers are there. Then, continue if you're interested!**

----

## Optional

If you managed to get your entire CKY-parser working and have an appetite for more, it might be fun to try it on some more sentences and grammars. Give the grammars below a try!

### Alternative Groucho-grammar

If you change the probabilities in the grammar, you'll get a different parse as the most likely one. Compare `groucho-grammar-1.txt` with `groucho-grammar-2.txt` and spot the difference in probabilities.

In [ ]:
## YOUR CODE HERE ##
# Read in the grammar
with open('groucho-grammar-2.txt') as F:
    grammar = PCFG(read_grammar_rules(F))
print("The grammar:\n", grammar)

The grammar:
 [S] -> [NP] [VP] (1.0)
[PP] -> [P] [NP] (1.0)
[NP] -> [Det] [N] (0.2)
[NP] -> [Det] [NP] (0.3)
[NP] -> [N] [PP] (0.3)
[NP] -> 'I' (0.2)
[VP] -> [V] [NP] (0.6)
[VP] -> [VP] [PP] (0.4)
[Det] -> 'an' (0.6)
[Det] -> 'my' (0.4)
[N] -> 'elephant' (0.5)
[N] -> 'pajamas' (0.5)
[V] -> 'shot' (1.0)
[P] -> 'in' (1.0)


### The man with the telescope

Another ambiguous sentence:

> I saw the man on the hill with the telescope.

A grammar for this sentence is specified in the file `telescope-grammar.txt`.

In [ ]:
## YOUR CODE HERE ##
# Read in the grammar
with open('telescope-grammar.txt') as F:
    grammar = PCFG(read_grammar_rules(F))
print("The grammar:\n", grammar)

The grammar:
 [S] -> [NP] [VP] (1.0)
[VP] -> [V] [NP] (0.6)
[VP] -> [VP] [PP] (0.4)
[NP] -> [NP] [PP] (0.3)
[NP] -> [Det] [N] (0.3)
[NP] -> [Det] [NP] (0.2)
[NP] -> 'I' (0.2)
[PP] -> [P] [NP] (0.8)
[PP] -> [PP] [PP] (0.2)
[V] -> 'saw' (1.0)
[Det] -> 'the' (1.0)
[N] -> 'man' (0.4)
[N] -> 'hill' (0.3)
[N] -> 'telescope' (0.3)
[P] -> 'on' (0.5)
[P] -> 'with' (0.5)


## Work description for Part 1

YOUR ANSWER HERE [100-200 words]

To solve ex1.1, we hard-coded the CKY algorithm, defining the lexical unary rules and assigning them a probability. Then we handled spans of increasing length, computing the probability of each rule and updating the score and backpointer charts accordingly.

For ex 1.2, we used the backpointer chart - filled in the previous exercise - to code the "build_tree". Starting from the root $<S>$, this function performs spanning recurrently, based on the probabilities on the backpointer chart.

Overall, the concepts were easy-to-understand and manageable Nevertheless, GenAI has been employed to treat arrays and matrices correctly.

# Part 2: Word sense disambiguation

In this part we will use the BERT transformer model's contextualized word embeddings to tackle the word sense disambiguation (WSD) task. The approach consists of the following:

1. Get the contextualized BERT embeddings for all tokens in a sense-annotated corpus;
2. For each sense $s$, calculate a mean vector of all the vectors of the words that are tagged with the sense $s$ in the training part of the corpus;
3. For each sense-annotated token $t$ in the test part of the corpus, assign $s$ sense to $t$ such that the vector of $s$ is the closest to the vector of $t$.
4. As a backup strategy for tokens in the test part for which no sense vector was obtained from the training part (i.e., tokens with unseen senses), use the 1st sense of the token by default.

## Setup

In [ ]:
# Course-specific package
! rm -rf assigntools
! git clone https://github.com/kovvalsky/assigntools.git
from assigntools.NLP.deep_learning import transformer_word2convec
from assigntools.M4LP.A1 import read_pickle, write_pickle

Cloning into 'assigntools'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 267 (delta 27), reused 6 (delta 3), pack-reused 200 (from 1)
Receiving objects: 100% (267/267), 66.36 KiB | 557.00 KiB/s, done.
Resolving deltas: 100% (129/129), done.


In [ ]:
import random, torch
import torch.nn.functional as F
from collections import defaultdict, Counter
from tqdm import tqdm
from tabulate import tabulate
import nltk
from more_itertools import chunked
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.corpus.reader.wordnet import Lemma
nltk.download('semcor')
nltk.download('wordnet')

# append any imports if needed

[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## SemCor

As a sense annotated corpus, we will use SemCor, conveniently available within NLTK. <code>semcor.sents()</code> iterates over all sentences represented as lists of tokens, while <code>semcor.tagged_sents()</code> iterates over the same sentences with additional annotation including WordNet Lemma identifiers (Lemma in WordNet stands for a particular sense of a word as opposed to a synset that is a set of Lemmas).

In [ ]:
# two sample sentence from the semcor corpus
# with their corresponding sense-annotated versions
for i in [0, 27]:
    print(semcor.sents()[i])
    print(semcor.tagged_sents(tag="sem")[i])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']]
['His', 'petition', 'charged', 'mental', 'cruelty

Let's prepare SemCor data for the disambiguation task. Since this is just an educational exercise and we don't aim at replicating the full results, we can use only a subset of SemCor. Take the first $N$ sentences of SemCor, pre-process the data, shuffle the sample in the data **randomly**, and finally split the data into the training and test sets.

In [ ]:
# Extract a part of the data for experiments
N = 10_000
semcor_annotated = list(semcor.tagged_sents(tag='sem')[:N])
semcor_tokenized = list(semcor.sents()[:N])
random.Random(42).shuffle(semcor_annotated)
random.Random(42).shuffle(semcor_tokenized)

## Ex2.1 [8pt] Preprocessing data

Create a function that takes as input a collection of sense-annotated sentences from SemCor and extracts the sense annotation. For each token of the sentence get either the corresponding WordNet sense or <code>None</code>.
<code>None</code> corresponds to tokens that are: (1) not annotated with a Lemma object sense (e.g. articles); (2) representing a part of a larger phrase that is annotated with a sense. The latter represents a simplification of the task.  
More info about NLTK's Lemma and Tree objects can be found here: [Lemma](https://www.nltk.org/api/nltk.corpus.reader.wordnet.html) and [Tree](https://www.nltk.org/api/nltk.tree.tree.html).

In [ ]:
from nltk.tree import Tree

def get_sns_annotations(data):
    """ data - sense tagged data from semcor
        return
            the sense annotations as a list of lists.
            The structure follows to semcor sentences and tokenization
            The elements of the list are None or a tuple of strings
            representing a synset and a lemma.
            None annotation means that a word token has no sense annotation
    """
    ### YOUR CODE HERE ###
    annotations = []

    for sentence in data:
        sentence_annotations = []

        for token in sentence:
            if isinstance(token, Tree):
                # If multi-word expression or named entity or complex tree structure, assign None to all tokens
                if len(token.leaves()) > 1 or isinstance(token.label(), str) or token.height() > 2:
                    sentence_annotations.extend([None] * len(token.leaves()))
                else:
                    # Single-token Tree with a Lemma label: extract annotation
                    lemma = token.label()
                    sentence_annotations.append((lemma.synset().name(), lemma.name()))
            else:
                # Plain token (string or list of strings): assign None for each token
                sentence_annotations.extend([None] * len(token))

        annotations.append(sentence_annotations)

    return annotations

In [ ]:
# TEST Ex2.1
semcor_senses = get_sns_annotations(semcor_annotated)

print("sample sentence:", semcor_tokenized[0])
print("sample annotation:", semcor_senses[0])

print("sample sentence:", semcor_tokenized[13])
print("sample annotation:", semcor_senses[13])

print("Total number of senses in the data =", len([ t for s in semcor_senses for t in s if t ]))

# test that for all sentences token number and annotation length are the same
for i, (senses, tokenized) in enumerate(zip(semcor_senses, semcor_tokenized, strict=True)):
    assert len(senses) == len(tokenized), \
        f"mismatch for {i}th sentence\n{senses}\n{tokenized}"

sample sentence: ['The', 'bronchial', 'artery', ',', 'except', 'for', 'a', 'small', 'number', 'of', 'short', 'branches', 'in', 'the', 'hilum', ',', 'contributes', 'none', 'of', 'the', 'pleural', 'blood', 'supply', '.']
sample annotation: [None, None, None, None, None, None, None, ('small.a.01', 'small'), ('number.n.02', 'number'), None, ('short.a.02', 'short'), ('branch.n.03', 'branch'), None, None, ('hilus.n.01', 'hilum'), None, ('contribute.v.02', 'contribute'), None, None, None, ('pleural.a.01', 'pleural'), ('blood.n.01', 'blood'), ('supply.n.01', 'supply'), None]
sample sentence: ['It', 'just', 'did', "n't", 'occur', 'to', 'Trig', 'that', 'anything', 'serious', 'would', 'happen', 'to', 'him', '.']
sample annotation: [None, ('merely.r.01', 'just'), None, None, ('occur.v.02', 'occur'), None, None, None, None, ('dangerous.s.02', 'serious'), None, ('happen.v.02', 'happen'), None, None, None]
Total number of senses in the data = 84296


Reference output:
```
sample sentence: ['The', 'bronchial', 'artery', ',', 'except', 'for', 'a', 'small', 'number', 'of', 'short', 'branches', 'in', 'the', 'hilum', ',', 'contributes', 'none', 'of', 'the', 'pleural', 'blood', 'supply', '.']
sample annotation: [None, None, None, None, None, None, None, ('small.a.01', 'small'), ('number.n.02', 'number'), None, ('short.a.02', 'short'), ('branch.n.03', 'branch'), None, None, ('hilus.n.01', 'hilum'), None, ('contribute.v.02', 'contribute'), None, None, None, ('pleural.a.01', 'pleural'), ('blood.n.01', 'blood'), ('supply.n.01', 'supply'), None]
sample sentence: ['It', 'just', 'did', "n't", 'occur', 'to', 'Trig', 'that', 'anything', 'serious', 'would', 'happen', 'to', 'him', '.']
sample annotation: [None, ('merely.r.01', 'just'), None, None, ('occur.v.02', 'occur'), None, None, None, None, ('dangerous.s.02', 'serious'), None, ('happen.v.02', 'happen'), None, None, None]
Total number of senses in the data = 84296
```

In [ ]:

# Run the following to get a reference data for comparison
# If your data differs from the reference, use the reference data in the subsequent parts
# !rm -f semcor_senses.pkl
# !wget -nv https://naturallogic.pro/_files_/download/mNLP/semcor_senses.pkl
# semcor_senses = read_pickle("semcor_senses.pkl")

In [ ]:
# create training and test sets
train_N = 9_000
semcor_X = {'train':semcor_tokenized[:train_N], 'test':semcor_tokenized[train_N:]}
semcor_Y = {'train':semcor_senses[:train_N], 'test':semcor_senses[train_N:]}

## BERT's contextualized vectors

After we have the training and test sets prepared with their gold sense annotations, it is time to get sense vectors for those senses that are occurring in the training set. Note that **contextualized vectors are crucial for the task** as a word (e.g., "book", "plant", "figure") can have different senses in different contexts.

We will use BERT transformer model to get contextualized word vectors for the words in the training and test sets. We will use the implementation of BERT in pytorch from the [transformers library](https://huggingface.co/docs/transformers/index).

Getting word vectors from BERT is not trivial as it uses a different type of tokenization than the traditional one. For example, the base-uncased version of BERT expects `Jupyter` tokenized as `ju`, `##py`, `##ter` while `Notebook` as `notebook` (note the lower casing of tokens due to the uncased version of BERT). To distinguish these two versions of tokenization and tokens, we will use `tokens` for BERT tokens and words for traditional tokenization. For example, the SemCor sentences use traditional tokenization.

If you want to learn more about BERT, [this](http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/) represents a gentle intro to BERT's wordpiece-based tokenizatio and contextualized word vectors.

In [ ]:
# if this cell errors with "A UTF-8 locale is required. Got ANSI_X3.4-1968"
# uncomment and run the next two lines
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# install transformer library
# !pip install transformers

In [ ]:
import transformers
from transformers import BertModel, AutoTokenizer
print(transformers.__version__) # 4.41.0

4.52.4


In [ ]:
# Load tokenizer (vocabulary)
MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# As usual, tokens are mapped to indices
print("The size of the token vocabulary", len(tokenizer.vocab))
for tok in ("dog", "##tion"):
    print(f"'{tok}' has index {tokenizer.vocab[tok]}")

for i in (3899, 3508):
    print(f"Reverse mapping: {i} --> {tokenizer.convert_ids_to_tokens(i)}")

The size of the token vocabulary 30522
'dog' has index 3899
'##tion' has index 3508
Reverse mapping: 3899 --> dog
Reverse mapping: 3508 --> ##tion


In [ ]:
example_input = "Transformers in Jupyter Notebook"
tok_result = tokenizer(example_input)
print("Output of a tokenizer: ", tok_result)
print("Tokens as word pieces: ", tokenizer.convert_ids_to_tokens(tok_result.input_ids))

Output of a tokenizer:  {'input_ids': [101, 19081, 1999, 18414, 7685, 3334, 14960, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
Tokens as word pieces:  ['[CLS]', 'transformers', 'in', 'ju', '##py', '##ter', 'notebook', '[SEP]']


`[CLS]` and `[SEP]` are special tokens use by BERT. `[CLS]` gets a vector that models the meaning of the entire input text sequence while `[SEP]` indicates sequence delimiters. Note that one of the tasks BERT was pre-trained on was guessing the next sentence, hence it was trained on sequence modeling, where elements of the sequence are sentences. Note that the output of `tokenizer([S1, S2])` and `tokenizer(S1, S2)` differ as in the first case the input is interpreted as a batch of two independent texts while in the second it is a sequence of texts.

The output of the tokenizer provides a sufficient input for BERT to process the input and assign contextualized embeddings.

In [ ]:
# Load pre-trained model (weights)
bert = BertModel.from_pretrained(MODEL_NAME)

#print parameters
bert.parameters

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): BertOutput(
          (dense): Linear(in_features=3072, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (pooler): BertPooler(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
  )
)>

In [ ]:
# let bert output hidden states
bert.config.output_hidden_states = True
# bert expects tensors as an input
tok_result = tokenizer(example_input, return_tensors='pt')
bert_output = bert(**tok_result)
print("Dimension of the last (12th) hidden states (batch size X token number X vector dim): ", bert_output.hidden_states[-1].shape)

Dimension of the last (12th) hidden states (batch size X token number X vector dim):  torch.Size([1, 8, 768])


Due to non-trivial correspondence between BERT tokens and words, we provide you with a ready function that takes a batch/list of word-tokenized sentences and processes them with the BERT model. In the end, it returns contextualized word vectors for each input word. The vector of the words that consist of several tokens is obtained by collating token vectors (e.g., taking the mean by default). The function allows to indicate from which layer the vectors should be extracted. For more details you can read the function definition [here](https://github.com/kovvalsky/assigntools/blob/main/NLP/deep_learning.py).

In [ ]:
# Batches with GPU accelerates the process ~30 times when used T4 GPU of colab compared to CPU
# Obviously for this toy example, efficiency doesn't matter
# this identifies whether GPU is present
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("we are using", device)

sample_batch = [ "Transformers in Jupyter Notebook".split(), "Transformers visited the earth".split() ]
sample_output = transformer_word2convec(bert, tokenizer, sample_batch, device=device, collate_tok_vec=torch.mean, layer=-1)

# illustrating the output
for sent in sample_output:
    for w in sent:
        print(f"'{w['word']}' tokenized as {w['tokens']} with tensor (first 3 components) {w['pt'][:3]}")
    print()

# Comparing vectors of two occurrences of "Transformers"
tvec1, tvec2 = sample_output[0][0]['pt'], sample_output[1][0]['pt']
print(f"{tvec1[:5]}... != {tvec2[:5]}...")
print(f"vectors cosine similarity = {F.cosine_similarity(tvec1, tvec2, dim=0)}")

we are using cuda
'Transformers' tokenized as ['transformers'] with tensor (first 3 components) tensor([-0.2651, -0.0888, -0.1850])
'in' tokenized as ['in'] with tensor (first 3 components) tensor([ 0.8255,  0.1081, -0.4694])
'Jupyter' tokenized as ['ju', '##py', '##ter'] with tensor (first 3 components) tensor([-0.1719, -0.5191,  0.3761])
'Notebook' tokenized as ['notebook'] with tensor (first 3 components) tensor([-0.2273, -0.4334,  0.7509])

'Transformers' tokenized as ['transformers'] with tensor (first 3 components) tensor([-0.4967,  0.0918, -0.0243])
'visited' tokenized as ['visited'] with tensor (first 3 components) tensor([ 1.2033, -0.1839, -0.5399])
'the' tokenized as ['the'] with tensor (first 3 components) tensor([-0.8393,  0.1038,  0.2743])
'earth' tokenized as ['earth'] with tensor (first 3 components) tensor([-0.8355, -0.2884, -0.1953])

tensor([-0.2651, -0.0888, -0.1850,  0.4485,  0.1538])... != tensor([-0.4967,  0.0918, -0.0243,  0.1993, -0.2419])...
vectors cosine simi

## Ex2.2 [10pt] Sense vectors

Process the training set with BERT using `transformer_word2convec` with default parameters. After getting word vectors, iterate over all train sentences, and for each sense, collect the word vectors. Note that words without sense annotations will be ignored in this process.

Since senses in the same WordNet synset are considered equivalent, we will be using synsets as sense labels. Prepare a dictionary with synset `(synset_str)` as a key and a single tensor as a value. The tensor should be a mean of all the word vectors collected for the sense (this is a default collating method used by `transformer_word2convec`).

This process is a time-consuming part of this assignment. It is recommended to use Colab's GPU: max 5min of T4 GPU will suffice to process all sentences with BERT. While processing the sentences, use batches of size 64 (Hint: `chunked` from `more_itertools` can do batching for you). Note that batches make a big difference with GPU. For the purposes of developing and debugging your solution, you may start by using a sample of 100 sentences, but then switch to the full training set.

In [ ]:
# you can reuse global vars bert and tokenizer
def get_sense2vec(data_X, data_Y, batch_size=64, device=device, collate=torch.mean):
    """ data_X and data_Y are a list of tokenized semcor sentences with their corresponding sense annotations.
        The last two arguments are the same as in transformer_word2convec
        Returns 2 dictionaries:
            Sense2VecList - { synset_str -> list of tensors  }
            Sense2AvgVec - { synset_str -> a mean tensor  }
    """
    ### YOUR CODE HERE ###
    sense2veclist = defaultdict(list)

    # Process data in batches
    for batch_X, batch_Y in zip(chunked(data_X, batch_size), chunked(data_Y, batch_size)):
        # Get BERT embeddings for the batch
        batch_embeddings = transformer_word2convec(bert, tokenizer, batch_X, device=device, collate_tok_vec=collate, layer=-1)

        # Process each sentence in the batch
        for sent_embeddings, sent_senses in zip(batch_embeddings, batch_Y):
            for word_embedding, sense in zip(sent_embeddings, sent_senses):
                if sense is not None:  # Only consider words with sense annotations
                    synset_str = sense[0]  # Get the synset string
                    sense2veclist[synset_str].append(word_embedding['pt'])

    sense2avgvec = {}  # Dictionary to store the average vector for each sense

    for synset, vectors in sense2veclist.items():
        # Stack the list of vectors into a single tensor (shape: [num_vectors, 768])
        stacked = torch.stack(vectors)

        # Compute the mean vector along the 0th dimension (i.e., average across all vectors)
        avg_vector = stacked.mean(dim=0)

        # Store the result in the dictionary
        sense2avgvec[synset] = avg_vector

    return sense2avgvec, sense2veclist

In [ ]:
%%time
# ~50min with CPU, less than 2min with T4 GPU
BATCH_SIZE = 64

Sense2AvgVec, Sense2VecList = get_sense2vec(semcor_X['train'], semcor_Y['train'], batch_size=BATCH_SIZE)

CPU times: user 1min 3s, sys: 357 ms, total: 1min 4s
Wall time: 1min 3s


In [ ]:

# If you want to use reference vectors for next exercises, run the following:
# !rm -f Sense2AvgVec.pkl
# !wget -nv https://naturallogic.pro/_files_/download/mNLP/Sense2AvgVec.pkl
# Sense2AvgVec = read_pickle("Sense2AvgVec.pkl")

In [ ]:
# TEST Ex2.2
for sns in [ 'mature.v.01', 'promptly.r.01', 'state.v.01', 'be.v.01']:
    assert isinstance(Sense2VecList[sns], list)
    assert isinstance(Sense2VecList[sns][0], torch.Tensor)
    assert isinstance(Sense2AvgVec[sns], torch.Tensor)
    print(f"{sns} sense has {len(Sense2VecList[sns])} vectors with the mean vector = {Sense2AvgVec[sns][:8]} ...")

mature.v.01 sense has 3 vectors with the mean vector = tensor([ 0.0587,  0.0471, -0.4079, -0.1624,  0.2848,  0.0910,  0.6782,  0.3704]) ...
promptly.r.01 sense has 6 vectors with the mean vector = tensor([-0.2618,  0.0485,  0.1117,  0.0635,  0.5795, -0.1821,  0.0408,  1.0724]) ...
state.v.01 sense has 410 vectors with the mean vector = tensor([ 0.3054,  0.2383, -0.0089, -0.0291,  0.2214,  0.0783,  0.2360,  0.5706]) ...
be.v.01 sense has 2465 vectors with the mean vector = tensor([ 0.0400,  0.0864,  0.0222, -0.0710,  0.2275, -0.0252,  0.2669,  0.5936]) ...


Reference output
```
mature.v.01 sense has 3 vectors with the mean vector = tensor([ 0.0587,  0.0471, -0.4079, -0.1624,  0.2848,  0.0910,  0.6782,  0.3704]) ...
promptly.r.01 sense has 6 vectors with the mean vector = tensor([-0.2618,  0.0485,  0.1117,  0.0635,  0.5795, -0.1821,  0.0408,  1.0724]) ...
state.v.01 sense has 410 vectors with the mean vector = tensor([ 0.3054,  0.2383, -0.0089, -0.0291,  0.2214,  0.0783,  0.2360,  0.5706]) ...
be.v.01 sense has 2465 vectors with the mean vector = tensor([ 0.0400,  0.0864,  0.0222, -0.0710,  0.2275, -0.0252,  0.2669,  0.5936]) ...
```

## Ex2.3 [12pt] WSD testing

Now we are going to evaluate the sense embeddings on the test set. Write a function that takes a list of tokenized sentences and a mask that indicates which words are supposed to get senses. The function should return the sense predictions aligned with the sentences. When predicting a sense for a word token, use the strategy outlined above, with 1st WordNet sense as a fallback. Here is the strategy in more detail:

- Use the sense vectors that were calculated based on the training set;
- For each sense-annotated word token $t$ (e.g. the verb `run`) in the test set, predict the synset $s$ (e.g., `'run.x.xx'`) such that the vector of $s$ is the closest to the contextualized vector of $t$ based on the cosine distance metric. Also make sure that the predicted sense is applicable to a word token using [`wn.synsets()`](https://www.nltk.org/howto/wordnet.html).
- There will be word tokens $t$ in the test set for which there won't be a sense vector collected from the training set, i.e., unseen senses. For such word tokens, us a backup strategy and predict the 1st sense of the word from WordNet, which is the most common sense of the word. This can be done using a built-in function from NLTK (e.g. <code>wn.lemmas('run')[0]</code>). For more info about NLTK's WordNet API, check [this](https://www.nltk.org/howto/wordnet.html).

Note that the info that a word token has a gold sense unseen in the training set (provided by the mask argument), is not realistic info as it presupposes knowledge about gold annotations.

Below you are provided with the sense masking that indicates whether a word token gets a sense and whether its sense was seen in the training set.

In [ ]:
# masking of annotations: None - has NO sense annotation; True - has sense annotation and the sense
# was seen in the training set; False - has sense annotation but the sense was NOT seen in the training set
test_sense_mask = [ [ i if i is None else ( True if i[0] in Sense2AvgVec else False ) for i in s ] \
                        for s in semcor_Y['test'] ]

def wsd_accuracy(predictions, reference, verbose=False):
    """ Calculates accuracy with respect to the word tokens that get sense annotations.
    """
    true_and_false = []
    for preds, refs in zip(predictions, reference, strict=True):
        for (p, r) in zip(preds, refs, strict=True):
            if r is not None:
                true_and_false.append(p == r[0])
                if verbose and p != r[0]:
                    print(f"wrong prediction ({p}) for ({r})")
    return sum(true_and_false)/len(true_and_false)

In [ ]:
def predict_senses(sense2vec, sentences, sense_mask, fallback=False,
                   batch_size=BATCH_SIZE, bert=bert, tokenizer=tokenizer, device=device, verbose=False):
    """ sense2vec - a dictioanry from synset strings to torch tensor vectors
        sentences - a list of sentences each being a list of word tokens
        sense_mask - it is aligned with tokens of sentences and tells if a token gets sense
                    and what type sense, seen or unseen in the training set.
        fallback - if True it uses first sense as the option for tokens with unseen senses.
        batch_size - the number of sentences is a batch
        bert, tokenizer - bert model and a tokenizer compatible with it
        device - a cpu or a gpu/cuda device that will be used by bert and tensor computations
        verbose - Prints whatever you want when it is False
        return predictions
            a list of list of predictions (None or a synset as a string) where the structure
            is aligned with the sentences
    """
    ### YOUR CODE HERE ###

    predictions = []

    # Process in batches
    for batch_X, batch_mask in zip(chunked(sentences, batch_size), chunked(sense_mask, batch_size)):
        # Get contextualized embeddings
        batch_embeddings = transformer_word2convec(
            bert, tokenizer, batch_X,
            device=device, collate_tok_vec=torch.mean, layer=-1
        )

        for sentence_vecs, mask_row, tokens in zip(batch_embeddings, batch_mask, batch_X):
            sentence_preds = []

            for token_vec, mask, token in zip(sentence_vecs, mask_row, tokens):
                if mask is None:
                    sentence_preds.append(None)
                    continue

                # Get possible senses of the token using WordNet
                possible_synsets = wn.synsets(token)

                if not possible_synsets:
                    sentence_preds.append(None)
                    continue

                # If the token has a seen sense (True in mask)
                if mask is True:
                    # Filter to possible synsets present in training
                    valid_synsets = [s.name() for s in possible_synsets if s.name() in sense2vec]

                    if not valid_synsets:
                        sentence_preds.append(None)
                        continue

                    # Stack all candidate sense vectors
                    synset_vecs = torch.stack([sense2vec[syn] for syn in valid_synsets]).to(device)
                    token_vec = token_vec['pt'].to(device)

                    # Compute cosine similarities
                    sims = F.cosine_similarity(token_vec.unsqueeze(0), synset_vecs)
                    best_idx = torch.argmax(sims).item()
                    sentence_preds.append(valid_synsets[best_idx])

                # If the token has an unseen sense (False in mask)
                elif fallback and mask is False:
                    # Use the most frequent WordNet sense
                    first_synset = possible_synsets[0].name()
                    sentence_preds.append(first_synset)
                else:
                    sentence_preds.append(None)

            predictions.append(sentence_preds)

    return predictions

In [ ]:
# TEST Ex2.3
# DON'T DELETE THE OUTPUT
# expected runtime less than a minute
predictions = predict_senses(Sense2AvgVec, semcor_X['test'], test_sense_mask)
predictions_fb_ssl = predict_senses(Sense2AvgVec, semcor_X['test'], test_sense_mask, fallback=True)

print("\nAccuracy of BERT      =", wsd_accuracy(predictions, semcor_Y['test']))
print("Accuracy of BERT + WN =", wsd_accuracy(predictions_fb_ssl, semcor_Y['test']))


Accuracy of BERT      = 0.6520353114271702
Accuracy of BERT + WN = 0.6972780774889652


Reference output (where `X<Y`):

```
Accuracy of BERT      = 0.6X...
Accuracy of BERT + WN = 0.6Y...
```

## Further experiments

Congratulations! You have reached the end of lab 4.

If you want an additional challenge, you can carry out further experiments on WSD task. Note that in the experiments we used the vectors from the last layer of BERT, but several research papers have shown that the vectors from different layers also encode useful information.

1.   You can test whether the vectors from other layers perform better than the last layer vectors.
2.   Word (and sense vectors) can be defined in terms of the combinations of the vectors from several layers of BERT. You can verify whether concatenating vectors from different layers (e.g., a concatenation of vectors from the last two layers) performs better than the vectors from the single layers.


## Work description for Part 2

YOUR ANSWER HERE [100-200 words]

The most challenging part was the `get_sns_annotations` function from Exercise 1.1, specifically getting the total number of senses in the data correct. Exercise 2.2 was pretty straightforward. Exercise 2.3 was more complicated, especially using the mask correctly for the fallback strategy. GenAI was used for understanding the exercises, debugging and refining functions.

# Part 3: Generalization & explainability

In [ ]:
# necessary to update datasets module for compatibility
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
# @title imports
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
print(f"transformers={transformers.__version__}")
import torch
print(f"torch={torch.__version__}")
import datasets
print(f"datasets={datasets.__version__}")
from datasets import load_dataset
import re

transformers=4.52.4
torch=2.6.0+cu124
datasets=3.6.0


## Task description

In this part, we will test one of the LLMs (which can be run in colab's modest environment) on the generalization and explainability while using chain-of-thought prompting ([Wei et al. 2022](https://arxiv.org/abs/2201.11903)).  
We will use an instruct model, a model that needs to be instructed with prompts to make predictions and doesn't need to be trained. So, we will use the model for inference and not for training or fine-tuning. Nonetheless, even Colab's least powerful GPU, such as T4, will be of practical use.  
Let's load an instruct model, in particular, `Phi-3-mini-4k-instruct` (see its [model card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) for details), and make a processing pipeline out of it. We use this model as it is optimal for the Colab environment and has very good results

In [ ]:
# @title model & tokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto", # Automatically place on GPU if available
    torch_dtype=torch.float16, # for good/efficient performance
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Create a pipeline
instruct_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
    max_new_tokens=512, # tokens to generate, doesn't count the prompt
    return_full_text=False, # don't return the input prompt
    do_sample=False, # for determinism, generates next token greedily
    use_cache=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# get model characteristics
def model_info(m):
    print(f"device: {next(m.parameters()).device}")
    print(f"dtype: {next(m.parameters()).dtype}")
    print(f"Att-impl: {m.config._attn_implementation}")
    print(f"Number of parameters: {sum(p.numel() for p in m.parameters())}")

model_info(model)

device: cuda:0
dtype: torch.float16
Att-impl: eager
Number of parameters: 3821079552


## Ex3.1 [7pt] Data generation

We will use the GSM8K dataset ([Cobbe et al. 2021](https://arxiv.org/pdf/2110.14168)) from OpenAI to access simple arithmetic math problems.

In [ ]:
# @title GSM8K
GSM8K = load_dataset("openai/gsm8k", 'main')

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
# we do some preprocessing, to get the short answer easily accessible
def preprocess_gsm8k(example):
    clean = re.sub('<<.*?>>', '', example['answer'])
    example['clean_answer'], example['short_answer'] = re.split('\s*\n####\s*', clean)
    return example

GSM8K = GSM8K.map(preprocess_gsm8k)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
# let's print a sample from the data
for k, v in GSM8K['train'][2025].items():
    print(f"{k}:\n{v}\n")

question:
At a bus station, a bus leaves every half-hour for 12 hours a day. How many buses leave the station for 5 days?

answer:
If one bus leaves every half-hour, then during one hour, there are 1 + 1 = <<1+1=2>>2 buses that leave the station.
During 5 days 2 * 12 * 5 = 120 buses leave the station.
#### 120

clean_answer:
If one bus leaves every half-hour, then during one hour, there are 1 + 1 = 2 buses that leave the station.
During 5 days 2 * 12 * 5 = 120 buses leave the station.

short_answer:
120



We need a prompt to give more elaborate instructions to the model. As the experiments show that the chain-of-thought (CoT) prompting boosts the model predictions, we are going to use it.  

Feel free to modify or use any of the two CoT templates given below.

In [ ]:
# @title Prompt templates
# Chain-of-Thought prompt template

# Feel free to define you CoT template

COT_TEMPLATE = """Solve the following problem step by step using chain-of-thought reasoning.

Problem: {problem}

Let me work through this step by step:"""

COT_TEMPLATE_DIALOG = """<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: {problem}
<|end|>

<|assistant|>: Let me work through this step by step:"""

# No Chain-of-Thought prompt template

NO_COT_TEMPLATE_DIALOG = """<|user|>:
Solve the following problem and provide only the answer:

Problem: {problem}
<|end|>

<|assistant|>: The correct answer is:"""

Here we are defining the substitution rules. The idea behind the experiment is to check whether concept and number replacements irrelevant from the reasoning perspective will affect the model's predictions.  
If the model correctly answers a math question `q`, then it should also correctly answer the math questions that are obtained from `q` with irrelevant concept/number substitutions.
Otherwise it will indicate the poor generalization capacity of the model.

Below you have an illustration of how to define the concept and number replacement rules for particular QA problems.

In [ ]:
# @title demo substitutions
problem_subs = [
    (2024, {'concept': [
                {'question': ['red -> pink', 'green -> yellow'],'short_answer': '30'},
                {'question': ['party -> event'],                'short_answer': '30'},
                {'question': ['balloons -> inflatable toys'],   'short_answer': '30'}
            ],
            'number': [
                {'question': ['20 -> 55', '15 -> 35'],  'short_answer': '85'},
                {'question': ['3 -> 15', '2 -> 15'],    'short_answer': '5'},
                {'question': ['20 -> 3', '15 -> 2'],    'short_answer': '0'}
                ]
    }),
    (2025, {'concept': [
                {'question': ['bus -> train', 'buses -> trains'],                           'short_answer': '120'},
                {'question': ['leaves -> arrives', 'leaves the -> arrives in the'],         'short_answer': '120'},
                {'question': ['day -> month', 'days -> months', 'half-hour -> half-day', 'hours -> days'],    'short_answer': '120'}
                ],
            'number': [
                {'question': ['12 -> 5'],               'short_answer': '50'},
                {'question': ['12 -> 23'],              'short_answer': '230'},
                {'question': ['12 -> 3', '5 -> 12'],    'short_answer': '72'}
                ]
    })
]

<font color="red">Pick **any three** QA problems from the test part of GSM8K.  
If your selected problems substantially overlap with other group's selected problems, this may be considered as plagarism and will be subject to penalization.</font>

For example, the chances of two groups selecting the same two problems are extremely low, taking into account that the test split contains >1300 problems.  

In [ ]:
# @title My substitutions
# don't change the variable name!!!
MY_SUBS = [
    ### YOUR CODE HERE ###
    # here should be three(!) problem IDs from the GSM8K['test']
    # each with corresponding three(!) concept and three(!) number replacement rules
    # So in total, with the mix=True, this should define 3x(3+3+3x3)=45 new QA problems
    # make sure that the generated problems have the correct gold standard answers

    # Problem ID 23: A candle melts by 2 centimeters every hour that it burns. How many centimeters shorter will a candle be after burning from 1:00 PM to 5:00 PM? Ans: 8
    (23, {'concept': [
                {'question': ['candle -> wax sculpture', 'melts -> shrinks'],      'short_answer': '8'},
                {'question': ['centimeters -> millimeters', 'shorter -> smaller'], 'short_answer': '8'},
                {'question': ['burns -> is lit', 'PM -> AM'],                      'short_answer': '8'}
            ],
            'number': [
                {'question': ['2 -> 3', '1:00 -> 2:00', '5:00 -> 7:00'],   'short_answer': '15'}, # 7-2=5 hours. 3 * 5 = 15
                {'question': ['2 -> 1.5', '1:00 -> 4:00', '5:00 -> 8:00'], 'short_answer': '6'},  # 13-9=4 hours. 1.5 * 4 = 6
                {'question': ['2 -> 4', '1:00 -> 10:00', '5:00 -> 11:00'], 'short_answer': '4'}   # 12-11=1 hours. 4 * 1 = 4
                ]
    }),

    # Problem ID 24: Kyle bought last year's best-selling book for $19.50. This is with a 25% discount from the original price. What was the original price of the book? Ans: 26
    (24, {'concept': [
                {'question': ['Kyle -> Sarah', 'book -> magazine'],                    'short_answer': '26'},
                {'question': ['best-selling -> most popular', 'bought -> purchased'],  'short_answer': '26'},
                {'question': ['discount -> price reduction', 'original price -> list price'], 'short_answer': '26'}
            ],
            'number': [
                {'question': ['19.50 -> 24.00', '25 -> 20'],  'short_answer': '30.0'}, # 24.00 / (1 - 0.20) = 24.00 / 0.8 = 30.0
                {'question': ['19.50 -> 30.00', '25 -> 40'],  'short_answer': '50.0'}, # 30.00 / (1 - 0.40) = 30.00 / 0.6 = 50.0
                {'question': ['19.50 -> 28.00', '25 -> 30'],  'short_answer': '40.0'} # 28.00 / (1 - 0.30) = 28.00 / 0.7 = 40.0
                ]
    }),

    # Problem ID 50: Lloyd has an egg farm. His chickens produce 252 eggs per day and he sells them for $2 per dozen. How much does Lloyd make on eggs per week? Ans: 294
    (50, {'concept': [
                {'question': ['egg -> apple', 'eggs -> apples', 'chickens -> trees'],                'short_answer': '294'},
                {'question': ['egg farm -> dairy farm', 'chickens -> cows', 'eggs -> milk bottles'], 'short_answer': '294'},
                {'question': ['produce -> lay', 'sells -> trades'],                                  'short_answer': '294'}
            ],
            'number': [
                {'question': ['252 -> 300', '2 -> 3'],      'short_answer': '525'},  # 300 apples/day * 7 days/week = 2100 apples/week.             2100/12 = 175 dozens. 175 * $3 = $525
                {'question': ['252 -> 144', '2 -> 2.50'],   'short_answer': '210'},  # 144 milk bottles/day * 7 days/week = 1008 milk bottles/week. 1008/12 = 84 dozens.  84 * $2.50 = $210
                {'question': ['252 -> 120', '2 -> 1.50'],   'short_answer': '105'}   # 120 eggs/day * 7 days/week = 840 eggs/week.                  840/12 = 70 dozens.   70 * $1.50 = $105
                ]
    })
]

In [ ]:
# @title generation function

def generate_prompted_samples(data, prob_subs, template=None, mix=True):
    """ The function takes data and a list of problem id and substitution rules.
        It returns a new list of QA problems that are obtained from the original ones
        by applying the substitutions.
        template is an optional flag telling the function to wrap new problems
        in a pre-defined template.
        mix tells the function to generate problems with a mixture of concept and number
        replacement rules. This option considers all possible combinations.
    """
    prompted_samples = [] # a lits of (id, mode, new_question, new_short_answer, q_subs)

    ### YOUR CODE HERE ###
    for prob_id, subs_dict in prob_subs:
        original_question = data[prob_id]['question']

        # 1. Concept substitutions
        for c_sub_info in subs_dict['concept']:
            new_question = original_question
            q_subs_applied = []
            for sub_pair in c_sub_info['question']:
                old, new = sub_pair.split(' -> ')
                # Use regex to replace whole words to avoid partial matches
                new_question = re.sub(r'\b' + re.escape(old) + r'\b', new, new_question, flags=re.IGNORECASE)
                q_subs_applied.append(sub_pair)
            final_question = template.format(problem=new_question) if template else new_question
            prompted_samples.append((prob_id, 'concept', final_question, c_sub_info['short_answer'], q_subs_applied))

        # 2. Number substitutions
        for n_sub_info in subs_dict['number']:
            new_question = original_question
            q_subs_applied = []
            for sub_pair in n_sub_info['question']:
                old, new = sub_pair.split(' -> ')
                # Use regex to replace whole numbers, ensuring they are not part of other numbers
                # e.g., '20' should not replace '200'
                new_question = re.sub(r'\b' + re.escape(old) + r'\b', new, new_question)
                q_subs_applied.append(sub_pair)
            final_question = template.format(problem=new_question) if template else new_question
            prompted_samples.append((prob_id, 'number', final_question, n_sub_info['short_answer'], q_subs_applied))

        # 3. Mixed substitutions (if mix is True)
        if mix:
            for c_sub_info in subs_dict['concept']:
                for n_sub_info in subs_dict['number']:
                    temp_question = original_question
                    q_subs_applied = []

                    # Apply concept substitutions first
                    for sub_pair in c_sub_info['question']:
                        old, new = sub_pair.split(' -> ')
                        temp_question = re.sub(r'\b' + re.escape(old) + r'\b', new, temp_question, flags=re.IGNORECASE)
                        q_subs_applied.append(sub_pair)

                    # Then apply number substitutions
                    for sub_pair in n_sub_info['question']:
                        old, new = sub_pair.split(' -> ')
                        temp_question = re.sub(r'\b' + re.escape(old) + r'\b', new, temp_question)
                        q_subs_applied.append(sub_pair)

                    # The short answer for mixed rules comes from the number substitution's calculation
                    final_question = template.format(problem=temp_question) if template else temp_question
                    prompted_samples.append((prob_id, 'mix', final_question, n_sub_info['short_answer'], q_subs_applied))

    return prompted_samples

In [ ]:
# TEST generate_prompted_samples
# Let's see how the function should work
prompted_samples = generate_prompted_samples(GSM8K['train'], problem_subs, template=COT_TEMPLATE_DIALOG)
print(f"total samples generated = {len(prompted_samples)}")

# print a sample generated problem
print(prompted_samples[-1][2])
print(f"Answer = {prompted_samples[-1][-2]}")

total samples generated = 30
<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?
<|end|>

<|assistant|>: Let me work through this step by step:
Answer = 72


The reference output of the above cell
```
total samples generated = 30
<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?
<|end|>

<|assistant|>: Let me work through this step by step:
Answer = 72
```

In [ ]:
# @title My generated problems
# don't change the variable names!!!

MY_PROBLEMS = generate_prompted_samples(GSM8K['test'], MY_SUBS, template=COT_TEMPLATE_DIALOG)
print(f"total samples generated = {len(MY_PROBLEMS)}")

# If you cannot generate the problems automatically, define them manually.
# Not a great solution (comes with penalties) but it will allow you to proceed with the next step

total samples generated = 45


## Ex3.2 [8pt] Evaluation

Now let's see how the model can be fed with an input and get predictions out of it.  
We will illustrate the both options, with the CoT template and without it.

In the end, you will have to run the instruct model on the generated samples with and without CoT prompts.

In [ ]:
# @title demo inference

CoT_prompt = '''
<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?
<|end|>

<|assistant|>: Let me work through this step by step:
'''
# note that the correct answer is 72
only_question = '''At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?'''
no_CoT_prompt = NO_COT_TEMPLATE_DIALOG.format(problem=only_question)

# @title generate_response
def generate_response(pipe_model, prompt, max_length=512):
    """Generate response using Hugging Face pipeline"""
    # Generate using pipeline
    outputs = pipe_model(
        prompt,
        max_new_tokens=max_length,
        pad_token_id=pipe_model.tokenizer.eos_token_id,
        eos_token_id=pipe_model.tokenizer.eos_token_id,
        return_full_text=False,  # Only return generated text, not the prompt
        use_cache=False
    )
    return outputs[0]['generated_text'].strip()

# response = generate_response(instruct_pipe, CoT_prompt, max_length=400)

print(f"{'CoT prompt':-^80}\n{CoT_prompt}\n{'':-^80}")
output = instruct_pipe(CoT_prompt, use_cache=False)
print(output[0]['generated_text'])
print(f'{"":=^80}')
print(f"{'No CoT prompt':-^80}\n{no_CoT_prompt}\n{'':-^80}")
output = instruct_pipe(no_CoT_prompt)
print(output[0]['generated_text'])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


-----------------------------------CoT prompt-----------------------------------

<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?
<|end|>

<|assistant|>: Let me work through this step by step:

--------------------------------------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



1. First, we need to determine how many buses leave the station in one day. Since a bus leaves every half-day, that means there are 2 buses leaving per day (one for the morning and one for the afternoon).

2. Next, we need to find out how many buses leave the station in a month. We know that buses leave for 3 days a month, so we multiply the number of buses per day (2) by the number of days in a month (3):

   2 buses/day * 3 days/month = 6 buses/month

3. Finally, we need to calculate how many buses leave the station in 12 months. We multiply the number of buses per month (6) by the number of months (12):

   6 buses/month * 12 months = 72 buses

So, 72 buses leave the station in 12 months.
---------------------------------No CoT prompt----------------------------------
<|user|>:
Solve the following problem and provide only the answer:

Problem: At a bus station, a bus leaves every half-day for 3 days a month. How many buses leave the station for 12 months?
<|end|>

<|assistant|>: Th

In [ ]:
# @title My inference
# Run the model on the generated problems with and without CoT prompting
# Feel free to include the problem generation script here too for no CoT prompting

### YOUR CODE HERE ###
def extract_raw_question_from_cot_prompt(modified_prompt):
    """Extracts the original question from a CoT-formatted prompt"""
    # This pattern matches everything between "Problem: " and the next prompt closing tag
    pattern = r'Problem:\s*(.*?)\s*<\|end\|>'
    match = re.search(pattern, modified_prompt, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

# Run the model on the generated problems with and without CoT prompting

cot_predictions = []
no_cot_predictions = []

# Define the original problem IDs selected
original_problem_ids = [23, 24, 50]

# --- Original Problem Inference ---
print("\n--- Running inference for ORIGINAL problems (CoT) ---")
for prob_id in original_problem_ids:
    original_data = GSM8K['test'][prob_id]
    original_question_text = original_data['question']
    gold_answer = original_data['short_answer']

    cot_orig_prompt = COT_TEMPLATE_DIALOG.format(problem=original_question_text)
    orig_cot_response = generate_response(instruct_pipe, cot_orig_prompt, max_length=512)

    cot_predictions.append({
        'prob_id': prob_id,
        'mode': 'original',
        'q_subs': [], # No substitutions
        'question': original_question_text,
        'prompt': cot_orig_prompt,
        'gold_answer': gold_answer,
        'prompt_type': 'CoT',
        'generated_text': orig_cot_response
    })
    print(f"Original CoT (ID: {prob_id}) done.")

print("\n--- Running inference for ORIGINAL problems (No-CoT) ---")
for prob_id in original_problem_ids:
    original_data = GSM8K['test'][prob_id]
    original_question_text = original_data['question']
    gold_answer = original_data['short_answer']

    no_cot_orig_prompt = NO_COT_TEMPLATE_DIALOG.format(problem=original_question_text)
    orig_no_cot_response = generate_response(instruct_pipe, no_cot_orig_prompt, max_length=512)

    no_cot_predictions.append({
        'prob_id': prob_id,
        'mode': 'original',
        'q_subs': [], # No substitutions
        'question': original_question_text,
        'prompt': no_cot_orig_prompt,
        'gold_answer': gold_answer,
        'prompt_type': 'No-CoT',
        'generated_text': orig_no_cot_response
    })
    print(f"Original No-CoT (ID: {prob_id}) done.")

# --- Substituted Problem Inference (existing code) ---
print("\n--- Running inference with CoT prompting on GENERATED problems ---")
for i, (prob_id, mode, modified_prompt, short_answer_gold, q_subs) in enumerate(MY_PROBLEMS):
    print(f"Processing CoT problem {i+1}/{len(MY_PROBLEMS)} (ID: {prob_id}, Mode: {mode})...")

    # Extract raw question from the CoT-formatted prompt
    raw_question = extract_raw_question_from_cot_prompt(modified_prompt)

    cot_response = generate_response(instruct_pipe, modified_prompt, max_length=512)

    cot_predictions.append({
        'prob_id': prob_id,
        'mode': mode,
        'q_subs': q_subs,
        'question': raw_question,
        'prompt': modified_prompt,
        'gold_answer': short_answer_gold,
        'prompt_type': 'CoT',
        'generated_text': cot_response
    })

print("\n--- Running inference without CoT prompting on GENERATED problems ---")
for i, (prob_id, mode, modified_prompt, short_answer_gold, q_subs) in enumerate(MY_PROBLEMS):
    print(f"Processing No-CoT problem {i+1}/{len(MY_PROBLEMS)} (ID: {prob_id}, Mode: {mode})...")

    # Extract raw question from the CoT-formatted prompt
    raw_question = extract_raw_question_from_cot_prompt(modified_prompt)

    no_cot_prompt = NO_COT_TEMPLATE_DIALOG.format(problem=raw_question)
    no_cot_response = generate_response(instruct_pipe, no_cot_prompt, max_length=512)

    no_cot_predictions.append({
        'prob_id': prob_id,
        'mode': mode,
        'q_subs': q_subs,
        'question': raw_question,
        'prompt': no_cot_prompt,
        'gold_answer': short_answer_gold,
        'prompt_type': 'No-CoT',
        'generated_text': no_cot_response
    })

print(f"\nCompleted inference for {len(cot_predictions)} CoT problems and {len(no_cot_predictions)} No-CoT problems.")

# Now, `cot_predictions` and `no_cot_predictions` will contain entries
# for both the original problems and the generated (substituted) problems.
# The 'mode' field will help distinguish them ('original', 'concept', 'number', 'mix').

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Running inference for ORIGINAL problems (CoT) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Original CoT (ID: 23) done.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Processing No-CoT problem 43/45 (ID: 50, Mode: mix)...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Processing No-CoT problem 44/45 (ID: 50, Mode: mix)...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Processing No-CoT problem 45/45 (ID: 50, Mode: mix)...

Completed inference for 48 CoT problems and 48 No-CoT problems.


In [ ]:
# It's helpful to group by original problem ID for assessment
grouped_predictions_for_assessment = {}
for p in cot_predictions + no_cot_predictions:
    if p['prob_id'] not in grouped_predictions_for_assessment:
        grouped_predictions_for_assessment[p['prob_id']] = {}
    if p['mode'] not in grouped_predictions_for_assessment[p['prob_id']]:
        grouped_predictions_for_assessment[p['prob_id']][p['mode']] = {
            'CoT': [],
            'No-CoT': []
        }
    grouped_predictions_for_assessment[p['prob_id']][p['mode']][p['prompt_type']].append(p)

print("\n" + "="*80)
print("             Manual Evaluation: CoT vs No-CoT Prompting            ")
print("="*80 + "\n")

# Sort by problem ID for consistent viewing
for prob_id in sorted(grouped_predictions_for_assessment.keys()):
    print(f"\n{'='*20} Original Problem ID: {prob_id} {'='*20}\n")

    # Access the 'original' mode first
    if 'original' in grouped_predictions_for_assessment[prob_id]:
        print(f"\n--- Original Problem ({prob_id}) ---\n")
        # Assuming there's one CoT and one No-CoT for original
        original_cot = grouped_predictions_for_assessment[prob_id]['original']['CoT'][0]
        original_no_cot = grouped_predictions_for_assessment[prob_id]['original']['No-CoT'][0]

        print(f"  Question: {original_cot['question']}")
        print(f"  Gold Answer: {original_cot['gold_answer']}\n")

        print("  --- CoT Prompt ---")
        print(original_cot['prompt'])
        print("\n  --- CoT Output ---")
        print(original_cot['generated_text'])

        print("\n  --- No-CoT Prompt ---")
        print(original_no_cot['prompt'])
        print("\n  --- No-CoT Output ---")
        print(original_no_cot['generated_text'])
        print("\n" + "-"*60 + "\n")

    # Then iterate through substituted modes
    for mode in ['concept', 'number', 'mix']:
        if mode in grouped_predictions_for_assessment[prob_id]:
            print(f"\n--- Mode: {mode.upper()} Substitutions for Problem {prob_id} ({len(grouped_predictions_for_assessment[prob_id][mode]['CoT'])} Samples) ---")
            for j in range(len(grouped_predictions_for_assessment[prob_id][mode]['CoT'])): # Iterate by index as they should be paired
                cot_pred = grouped_predictions_for_assessment[prob_id][mode]['CoT'][j]
                no_cot_pred = grouped_predictions_for_assessment[prob_id][mode]['No-CoT'][j]

                print(f"\n  Sample {j+1}:")
                print(f"    Substitutions Applied: {cot_pred['q_subs']}")
                print(f"    Question: {cot_pred['question']}") # Raw question (after subs, before prompt template)
                print(f"    Gold Answer: {cot_pred['gold_answer']}\n")

                print("    --- CoT Prompt ---")
                print(cot_pred['prompt'])
                print("\n    --- CoT Output ---")
                print(cot_pred['generated_text'])

                print("\n    --- No-CoT Prompt ---")
                print(no_cot_pred['prompt'])
                print("\n    --- No-CoT Output ---")
                print(no_cot_pred['generated_text'])

                print("-" * 50) # Separator for individual samples


             Manual Evaluation: CoT vs No-CoT Prompting            


==================== Original Problem ID: 23 ====================


--- Original Problem (23) ---

  Question: A candle melts by 2 centimeters every hour that it burns. How many centimeters shorter will a candle be after burning from 1:00 PM to 5:00 PM?
  Gold Answer: 8

  --- CoT Prompt ---
<|user|>:
Solve the following problem step by step using chain-of-thought reasoning:

Problem: A candle melts by 2 centimeters every hour that it burns. How many centimeters shorter will a candle be after burning from 1:00 PM to 5:00 PM?
<|end|>

<|assistant|>: Let me work through this step by step:

  --- CoT Output ---
1. First, we need to determine the total time the candle burns. The candle starts burning at 1:00 PM and ends at 5:00 PM.
2. To find the duration, we subtract the start time from the end time: 5:00 PM - 1:00 PM = 4 hours.
3. Now that we know the candle burns for 4 hours, we can calculate how much it melts during

### Evaluation results

<font color="red">█████ FILL IN THE TABLE █████</font>

<font color="red">Replace the dummy numbers while preserving the formatting.</font>  

The 0s should be replaced with the number of correct predictions. For Concept and Number types, this is a maximum of 3, as the total number of generated samples is 3 per type. For the Mix type, it will be between 0 and 9.
You are also required to provide an evaluation on the original problems too.

You are expected to manually fill in the table with correct counts because you will have to manually assess the correctness of CoT reasoning/explanation. Expl.+Ans means that both explanation text and the answer should be correct.
It is recommended that group members divide the assessment task among each other.

Note that counting correct answers is as simple as one needs to manually spotting the final numerical answer and comparing it to the correct number. It is important to print the results in a way that will make it easy for you to assess explanations and the numerical answers.

---
### Table for the model with CoT

The model gave correct answers and explanations in all cases.

<table style="border: 1px solid black; border-collapse: separate; width: 100%;">
  <thead>
    <tr style="text-align: center; border: 1px solid black;">
      <th style="border: 1px solid black;">pro id</th>
      <th style="border: 1px solid black; width: 200px;">question</th>
      <th style="border: 1px solid black;">Ans</th>
      <th style="border: 1px solid black;" colspan="2">#Corr. Original (1)</th>
      <th style="border: 1px solid black;" colspan="2">#Corr. Concept (3)</th>
      <th style="border: 1px solid black;" colspan="2">#Corr. Number (3)</th>
      <th style="border: 1px solid black;" colspan="2">#Corr. Mix (9)</th>
    </tr>
    <tr style="text-align: center; border: 1px solid black;">
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Expl.+Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Expl.+Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Expl.+Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Expl.+Ans.</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>23</td>
      <td style="width: 200px;">A candle melts by 2 centimeters every hour that it burns.<br>How many centimeters shorter will a candle be after burning<br>from 1:00 PM to 5:00 PM?</td>
      <td>8</td>
      <td>1</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>9</td>
      <td>9</td>
    </tr>
    <tr>
      <td>24</td>
      <td style="width: 200px;">Kyle bought last year's best-selling book for $19.50. This is<br>with a 25% discount from the original price. What was the<br>original price of the book?</td>
      <td>26</td>
      <td>1</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>9</td>
      <td>9</td>
    </tr>
    <tr>
      <td>50</td>
      <td style="width: 200px;">Lloyd has an egg farm. His chickens produce 252 eggs per<br>day and he sells them for $2 per dozen. How much does<br>Lloyd make on eggs per week?</td>
      <td>294</td>
      <td>1</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
      <td>9</td>
      <td>9</td>
    </tr>
  </tbody>
</table>

---
### Table for the model with <font color="red">No</font> CoT

<table style="border: 1px solid black; border-collapse: separate; width: 100%;">
  <thead>
    <tr style="text-align: center; border: 1px solid black;">
      <th style="border: 1px solid black;">pro id</th>
      <th style="border: 1px solid black; width: 200px;">question</th>
      <th style="border: 1px solid black;">Ans</th>
      <th style="border: 1px solid black;">#Corr. Original (1)</th>
      <th style="border: 1px solid black;">#Corr. Concept (3)</th>
      <th style="border: 1px solid black;">#Corr. Number (3)</th>
      <th style="border: 1px solid black;">#Corr. Mix (9)</th>
    </tr>
    <tr style="text-align: center; border: 1px solid black;">
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;"></th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
      <th style="border: 1px solid black;">Ans.</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>23</td>
      <td style="width: 200px;">A candle melts by 2 centimeters every hour that it burns.<br>How many centimeters shorter will a candle be after burning<br>from 1:00 PM to 5:00 PM?</td>
      <td>8</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
    </tr>
    <tr>
      <td>24</td>
      <td style="width: 200px;">Kyle bought last year's best-selling book for $19.50. This is<br>with a 25% discount from the original price. What was the<br>original price of the book?</td>
      <td>26</td>
      <td>1</td>
      <td>3</td>
      <td>3</td>
      <td>3</td>
    </tr>
    <tr>
      <td>50</td>
      <td style="width: 200px;">Lloyd has an egg farm. His chickens produce 252 eggs per<br>day and he sells them for $2 per dozen. How much does<br>Lloyd make on eggs per week?</td>
      <td>294</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

For No-CoT prompting, the model gave only correct anwsers for the original and modified questions for the problems 23 and 24 but struggled with the original and modified questions for problem 50.

In all cases for problem 50, it first gave a short immediate answer as instructed and then it provided an explanation for its answer even though it was instructed to not do so. Sometimes, the explanation ended up giving a different answer which in most cases was corrected, contradicting the original short answer it gave in the beginning.

Specifically, for three cases (the original question, the second 'concept' modified question and the 'mixed' modified question), it gave a wrong answer even after the explanation. In all other cases, the explanation corrected the initial wrong answer.

## Discussion

Based on the results in the table, discuss the generalization capacity of the model, the contribution of the CoT prompting, and the alignment of the explanations with the predicted answers.

<font color="red">█████ ANSWER UNDER THIS LINE [100-200 words] █████</font>

The results for Chain-of-Thought (CoT) prompting indicate that the model exhibits strong generalization capacity across varied reasoning tasks, maintaining consistent performance on unseen examples.

CoT provides accurate results and also contributes to higher interpretability, as the step-by-step explanations offer transparency into the model's decision-making process.

Importantly, the explanations always aligned with the predicted answers, reinforcing confidence in the model's internal consistency.

Overall, CoT prompting helps the model think step by step and also makes it easier to assess its reasoning.

---

In contrast, the No-CoT prompting strategy revealed limitations. While problems 23 and 24 were solved correctly, problem 50 consistently posed challenges, with the model giving incorrect answers and generating explanations despite being instructed otherwise.

This tendency for unprompted explanations highlights a lack of strict adherence to instructions and potential internal inconsistencies without explicit step-by-step prompting.

Interestingly, these explanations often contradicted the initial short answer and, in most cases, corrected it. This behavior suggests that the model needs intermediate reasoning steps to arrive at the correct conclusion, reinforcing the importance of CoT prompting for both accuracy and consistency.

The significant performance drop on Problem 50 without CoT indicates that CoT prompting is crucial for unlocking the model's full reasoning potential and improving its accuracy and instruction following on more complex or multi-step problems, thereby enhancing its generalization and reliability.

## Work description for Part 3

YOUR ANSWER HERE [100-200 words]

The most challenging part was ensuring substitutions, especially number changes, were done correctly. The manual evaluation and interpretation of the results was straightforward but time-consuming. GenAI was used for refining regex for substitutions and debugging.

# Acknowledgments

Most of this part 1 was developed by Joost Bastings. Later it was revised by Tejaswini Deoskar.  
The recent updates by Lasha Abzianidze make the notebook more streamlined and foolproof from the grading and the large course perspectives.

The initial version of Part 2 by Denis Paperno was a replication of the WSD experiment for ELMo. The assignment was built around the allennlp library.
Since 2022-23 course, the assignment was substantially changed by Lasha Abzianidze. allennlp was replaced with pytorch and transformers library. ELMo was replaced with BERT.

Part 3 is a new addition to 2024-25 notebook, by Lasha Abzianidze.